# Diseño de acople sintonizado

In [1]:
import sympy as sp
from sympy.core.compatibility import iterable
import numpy as np

## Parametros

In [2]:
# frec de resonancia, 1(ultimoNumDNI) MHz
fo=16E6
# ancho de banda, 10%fo
BW=10*fo/100
# impedancias
Ra=50
RL=1E3

# asumir una CT inicial para cumplir la fo, tratando de crear un inductor cuadrado
# CT=188E-12

# 310, 250, 200, 150, 169
CT=169E-12

print(f"requerimientos:")
print(f"\tfrec de resonancia = {fo/1E6} MHz, BW = {BW/1E6} MHz")
print(f"\timpedancias entrada/salida {Ra} Ohm/{RL/1E3} kOhm\n")
print(f"se adopta un CT inicial de {CT/1E-12} pF")

requerimientos:
	frec de resonancia = 16.0 MHz, BW = 1.6 MHz
	impedancias entrada/salida 50 Ohm/1.0 kOhm

se adopta un CT inicial de 169.00000000000003 pF


## Sacar L (teorico)

In [3]:
# despejando LT
""" 1/(4 * pi^2 * CT * fo^2) """

LT=1/(4*np.pi**2 * CT * fo**2)

print(f"L(teorico) = {LT/1E-6} uH.")

# pasar inductancia a [uH]
LT=LT/1E-6

L(teorico) = 0.5854820615427249 uH.


## Diseño del inductor (curvas J/K)

In [24]:
# diametro del cobre [mm]
Dcu=1.2E-3
# diametro del nucleo de aire [cm]
Dnuc=1

# diametro total del inductor [cm]
Dind=Dnuc+Dcu/1E-2

print(f"diseñar inductor de {LT} uH:")
print(f"\tnucleo de {Dnuc} cm y cobre de {Dcu/1E-3} mm")
print(f"\tdiametro medio del inductor de {Dind} cm.\n")

# criterio de separacion optima de Butterworth [vue/cm]
""" Nsop = 1/(1.41 * Dcu[cm])"""
Nsop=1/(1.41 * (Dcu/1E-2))
# para armarlo se separa con el mismo alambre y se apreta hasta llegar al largo calculado

# ----------------------------------------------------------------------------------
# diseño, ecuacion K de nagaoka
""" L[uH] = (Dind[cm])^3 * (Nsop[vue/cm])^2 * K * 1E-3 """
print("-"*60)
print("Diseño, ecuacion K de Nagaoka:")
print("-"*60)

""" K == 1000.0 * LT/((Dind[cm])^3 * (Nsop[vue/cm]^2)) """
K=1000.0*LT/((Dind**3) * (Nsop**2))

print(f"\tcoeficiente K: {K} ({np.log10(K)} en log).\n")

# entrar a grafica K
relLD=np.float32(input("dar relacion l/D de la tabla >> "))

print(f"\tentrando al grafico K, nos da la relacion l/Dind: {relLD} aprox.\n")

# largo del inductor teorico
l=relLD*Dind
# numero de espiras teorico
N=Nsop*l

print(f"\ttenemos un inductor de {l} cm de largo y {N} vueltas.\n")

# correjir numero de espiras
N=np.float32(input("redondear numero de vueltas >> "))

# correjir el nuevo largo y relacion l/Dind
l=N/Nsop
relLD=l/Dind

print(f"\tcorrijiendo a {N} espiras, tenemos la nueva relacion l/Dind {relLD}.\n")

# ----------------------------------------------------------------------------------
# verificacion, ecuacion J de Nagaoka
print(f"{'-'*60} \nverificacion, ecuacion J de Nagaoka: \n{'-'*60}")

J=np.float32(input(f"dar J para relacion l/D = {relLD} (log {np.log10(relLD)}) > "))

""" L[uH] = N^2 * (Dind [cm])/2 * J * 1E-3 """
LT=(N**2) * Dind/2 * J * 1E-3

print(f"\tentrando al grafico J para la relacion {relLD}\n")
print(f"\tqueda un inductor de {LT} uH.")

# inductancia teorica
""" L=(uo * Aind * N^2)/l """
Aind=np.pi*((Dind*1E-2)/2)**2

print(f"\tde inductancia teorica: {( ((4*np.pi*1E-7) * Aind * N**2)/(l*1E-2) )/1E-6} uH\n")

# resultados
print("-"*60)
print("resultados del diseño:")
print("-"*60)

print(f"inductor de {LT} uH aprox.")
print(f"\tcon nucleo de {Dnuc} cm, largo de {l} cm, diametro de cobre de {Dcu/1E-3} mm")
print(f"\tcon {N} espiras y relacion de forma {relLD}")

# inductancia a [H]
LT=LT*1E-6

diseñar inductor de 0.5854820615427249 uH:
	nucleo de 1 cm y cobre de 1.2 mm
	diametro medio del inductor de 1.1199999999999999 cm.

------------------------------------------------------------
Diseño, ecuacion K de Nagaoka:
------------------------------------------------------------
	coeficiente K: 11.930543890053094 (1.0766602427537115 en log).

	entrando al grafico K, nos da la relacion l/Dind: 1.5 aprox.

	tenemos un inductor de 1.6799999999999997 cm de largo y 9.929078014184396 vueltas.

	corrijiendo a 10.0 espiras, tenemos la nueva relacion l/Dind 1.5107142857142857.

------------------------------------------------------------ 
verificacion, ecuacion J de Nagaoka: 
------------------------------------------------------------
	entrando al grafico J para la relacion 1.5107142857142857

	queda un inductor de 0.5319999999999999 uH.
	de inductancia teorica: 0.7317040047710694 uH

------------------------------------------------------------
resultados del diseño:
--------------------

## Correjir C

In [25]:
# ajustar nuevo valor de C
""" CT == 1/(4 * pi^2 * LT * fo^2) """

CT=1/(4* (np.pi**2) * LT * (fo**2))

print(f"correjido a CT = {CT/1E-12} pF.\n")

correjido a CT = 185.98960225699346 pF.



## Calcular R de perdidas segun nagaoka

In [26]:
# Q decargado del inductor
""" Qd == 8850 * sqrt(fo[MHz]) * (Dind[cm] * l[cm])/(102*l[cm] + 45*Dind[cm]) """
Qd=8850 * np.sqrt(fo/1E6) * (Dind*l)/(102*l + 45*Dind)

# resistencia de perdida
""" Rp = Qd * 2*pi*fo * L """
Rp=Qd * 2*np.pi*fo * LT

print(f"resistencia de perdidas {Rp/1E3} KOhm\n")

resistencia de perdidas 16.090125255151797 KOhm



## Sacar resistencia total

In [27]:
# factor Q para RLC en paralelo es inverso al RLC serie
""" Q == RT/XLo == fo/BW"""
Qparal=fo/BW

# resistencia total para este BW
RT=Qparal * 2*np.pi*fo * LT

print(f"para un Qparalelo = {Qparal}, tenemos una RT de {RT}")

para un Qparalelo = 10.0, tenemos una RT de 534.8247333471262


## Buscar capacitores C1-4

In [28]:
# transformando circuito y reflejando al primario
""" RT == rrRa // rrRL // Rp """
"""     rrRa == Ra * (1 + C2/C1)^2 == 2*RT """
"""     rrRL // Rp == 1/(1/rrRL + 1/Rp) == 2*RT"""
"""         rrRL == RL * (1 + C4/C3)^2 """

# capacidad total
""" CT == C1///C2 + C3///C4 == CT/2 + CT/2 """

# sistema de ecuaciones, buscar C1 y C2
""" C1///C2 == 1/(1/C1 + 1/C2) == CT/2 """
""" rrRa == Ra * (1 + C2/C1)^2 == 2*RT """

# resolviendo a mano
""" C2 == CT/2 * sqrt(2*RT/Ra) """
""" C1 == C2/(sqrt(2*RT/Ra) - 1) """

C2=CT/2 * np.sqrt(2*RT/Ra)
C1=C2/(np.sqrt(2*RT/Ra) - 1)


# sistema de ecuaciones, buscar C3 y C4
""" C3///C4 == 1/(1/C3 + 1/C4) == CT/2 """
""" rrRL // Rp == 1/(1/rrRL + 1/Rp) == 2*RT"""
"""     rrRL == RL * (1 + C4/C3)^2 """

# resolviendo a mano
""" rrRL == 2*RT/(1 - 2*RT/Rp) """
""" C4 == CT/2 * sqrt(rrRL/RL) """
""" C3 == C4/(sqrt(rrRL/RL) - 1) """

rrRL=2*RT/(1 - 2*RT/Rp)
C4=CT/2 * np.sqrt(rrRL/RL)
C3=C4/(np.sqrt(rrRL/RL) - 1)

print("resolviendo:")
print(f"\tC1 = {C1/1E-12} pF")
print(f"\tC2 = {C2/1E-12} pF")

print(f"\tverif: C1///C2 == CT/2? ({1/(1/C1 + 1/C2)} == {CT/2}), rrRa == 2*RT? ({Ra * (1 + C2/C1)**2} == {2*RT})\n")

print(f"\tC3 = {C3/1E-12} pF")
print(f"\tC4 = {C4/1E-12} pF")

print(f"\tverif: C3///C4 == CT/2? ({1/(1/C3 + 1/C4)} == {CT/2}), rrRL//Rp == 2*RT? ({1/(1/Rp + 1/((RL * (1 + C4/C3)**2)))} == {2*RT})\n")

resolviendo:
	C1 = 118.6467306684989 pF
	C2 = 430.1247243743381 pF
	verif: C1///C2 == CT/2? (9.299480112849674e-11 == 9.299480112849674e-11), rrRa == 2*RT? (1069.6494666942526 == 1069.6494666942524)

	C3 = 1413.365538290862 pF
	C4 = 99.54450174933473 pF
	verif: C3///C4 == CT/2? (9.299480112849675e-11 == 9.299480112849674e-11), rrRL//Rp == 2*RT? (1069.6494666942526 == 1069.6494666942524)



## Normalizar componentes

In [29]:
# si se normalizan las capacidades
# C1n=120E-12
# C2n=390E-12 + 47E-12
# C3n=1800E-12
# C4n=100E-12

C1n=120E-12
C2n=(390 + 47)*1E-12
C3n=1500*1E-12
C4n=100E-12

# capacidad total
CTn=(1/(1/C1n + 1/C2n)) + (1/(1/C3n + 1/C4n))
print(f"capacidad normalizado total {CTn/1E-12} pF")

# resonando a 
fon=1/(2*np.pi * np.sqrt(LT*CTn))
print(f"\tresonando a {fon/1E6} MHz")

# factor Qparal
rrRa=Ra*(1 + C2n/C1n)**2
rrRL=RL*(1 + C4n/C3n)**2
RT=1/(1/Rp + 1/rrRL + 1/rrRa)

Qparal=RT/(2*np.pi*fon * LT)
print(f"\tfactor de merito {Qparal}")

capacidad normalizado total 187.89721723518852 pF
	resonando a 15.918573293329299 MHz
	factor de merito 10.05345659771558


### ECUACIONES RELOCAS AUXILIARES

In [22]:
# frecuencia de resonancia, sacar LT
""" fo = 1/(2*pi * sqrt(LT*CT)) """
ssfo,ssLT,ssCT=sp.symbols("ssfo,ssLT,ssCT")

ec0=sp.Eq(ssfo,1/(2*sp.pi*sp.sqrt(ssLT*ssCT)))
sp.pprint(ec0)
sp.print_latex(ec0)

print(f"\nresolviendo, LT == {sp.solve(ec0,ssLT)}\n")
print("-"*60)
# ----------------------------------------------------------------------------

# ecuacion de nagaoka, sacar K
""" L[uH] = (Dind[cm])^3 * (Nsop[vue/cm])^2 * K * 1E-3 """
ssLT,ssDind,ssNsop,ssK=sp.symbols("ssLT,ssDind,ssNsop,ssK")

ec1=sp.Eq(ssLT,ssDind**3 * ssNsop**2 * ssK * 1E-3)
sp.pprint(ec1)

print(f"\nresolviendo, K == {sp.solve(ec1,ssK)}\n")
print("-"*60)
# ----------------------------------------------------------------------------

# frecuencia de resonancia, sacar CT
""" fo = 1/(2*pi * sqrt(LT*CT)) """
sp.pprint(ec0)

print(f"\nresolviendo, CT == {sp.solve(ec0,ssCT)}\n")
print("-"*60)
# ----------------------------------------------------------------------------

# transformando circuito y reflejando al primario
""" RT == rrRa // rrRL // Rp """
"""     rrRa == Ra * (1 + C2/C1)^2 == 2*RT """
"""     rrRL // Rp == 1/(1/rrRL + 1/Rp) == 2*RT"""
"""         rrRL == RL * (1 + C4/C3)^2 """

ssrrRa,ssRa,ssC2,ssC1=sp.symbols("ssrrRa,ssRa,ssC2,ssC1")
ec1=sp.Eq(ssrrRa,ssRa * (1 + ssC2/ssC1)**2)
sp.print_latex(ec1)

ssrrRL,ssRL,ssC4,ssC3=sp.symbols("ssrrRL,ssRL,ssC4,ssC3")
ec2=sp.Eq(ssrrRL,ssRL * (1 + ssC4/ssC3)**2)
sp.print_latex(ec2)

print("-"*60)

# capacidad total
""" CT == C1///C2 + C3///C4 == CT/2 + CT/2 """

# sistema de ecuaciones, buscar C1 y C2
""" C1///C2 == 1/(1/C1 + 1/C2) == CT/2 """
""" rrRa == Ra * (1 + C2/C1)^2 == 2*RT """

# resolviendo a mano
""" C2 == CT/2 * sqrt(2*RT/Ra) """
""" C1 == C2/(sqrt(2*RT/Ra) - 1) """
ssCT,ssRT=sp.symbols("ssCT,ssRT")
ec3=sp.Eq(ssC2,ssCT/2 * sp.sqrt(2*ssRT/ssRa))
ec4=sp.Eq(ssC1,ssC2/(sp.sqrt(2*ssRT/ssRa) - 1))
sp.print_latex(ec3)
sp.print_latex(ec4)


# sistema de ecuaciones, buscar C3 y C4
""" C3///C4 == 1/(1/C3 + 1/C4) == CT/2 """
""" rrRL // Rp == 1/(1/rrRL + 1/Rp) == 2*RT"""
"""     rrRL == RL * (1 + C4/C3)^2 """

# resolviendo a mano
""" rrRL == 2*RT/(1 - 2*RT/Rp) """
""" C4 == CT/2 * sqrt(rrRL/RL) """
""" C3 == C4/(sqrt(rrRL/RL) - 1) """
ssRp=sp.symbols("ssRp")
ec5=sp.Eq(ssrrRL,2*ssRT/(1 - 2*ssRT/ssRp))
sp.print_latex(ec5)
print("-"*60)

# ---------------------------------------------------------------------
"""L[uH] = (Dind[cm])^3 * (Nsop[vue/cm])^2 * K * 1E-3"""
ssL,ssDind,ssNsop,ssK=sp.symbols("ssL,ssDind,ssNsop,ssK")
ec6=sp.Eq(ssL,ssDind**3 * ssNsop**2 * ssK * 1E-3)
sp.print_latex(ec6)
print("-"*60)


               1        
ssfo = ─────────────────
             ___________
       2⋅π⋅╲╱ ssCT⋅ssLT 
ssfo = \frac{1}{2 \pi \sqrt{ssCT ssLT}}

resolviendo, LT == [1/(4*pi**2*ssCT*ssfo**2)]

------------------------------------------------------------
                   3           2
ssLT = 0.001⋅ssDind ⋅ssK⋅ssNsop 

resolviendo, K == [1000.0*ssLT/(ssDind**3*ssNsop**2)]

------------------------------------------------------------
               1        
ssfo = ─────────────────
             ___________
       2⋅π⋅╲╱ ssCT⋅ssLT 

resolviendo, CT == [1/(4*pi**2*ssLT*ssfo**2)]

------------------------------------------------------------
ssrrRa = ssRa \left(1 + \frac{ssC_{2}}{ssC_{1}}\right)^{2}
ssrrRL = ssRL \left(1 + \frac{ssC_{4}}{ssC_{3}}\right)^{2}
------------------------------------------------------------
ssC_{2} = \frac{\sqrt{2} ssCT \sqrt{\frac{ssRT}{ssRa}}}{2}
ssC_{1} = \frac{ssC_{2}}{\sqrt{2} \sqrt{\frac{ssRT}{ssRa}} - 1}
ssrrRL = \frac{2 ssRT}{- \frac{2 ssRT}{ssRp} + 1}
-------

In [21]:
# imports
import matplotlib.pyplot as plt
%matplotlib qt5

# grafico de la respuesta

fax=np.arange(12.1,13.3,.1)
amp=[2.6,3,3.4,3.7,4,4.2,4.23,4.1,3.8,3.6,3.2,3,2.6]
amp=np.array(amp)

# amp a dB
amp=20*np.log10(amp/2)

plt.figure()
plt.title("Respuesta en frecuencia.")
plt.plot(fax,amp,label="Respuesta")
plt.xlabel("f (MHz)")
plt.ylabel("vo/vi (dB)")
plt.hlines(np.max(amp)-3,np.min(fax),np.max(fax),color="r",label="BW")
plt.grid()
plt.legend()

plt.show()